In [ ]:
import os
from pathlib import Path
from tqdm import tqdm#_notebook as tqdm
from pku_autonomous_driving import io, util, dataset, resnet, centernet, prediction, graphics

import importlib
importlib.reload(io)
importlib.reload(util)
importlib.reload(dataset)
importlib.reload(resnet)
importlib.reload(centernet)
importlib.reload(prediction)
importlib.reload(graphics)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torchvision
from pku_autonomous_driving.transform import CropBottomHalf, PadByMean, Resize, Normalize, DropPointsAtOutOfScreen, CreateMask, CreateRegr, ToCHWOrder
from pku_autonomous_driving.const import IMG_WIDTH, IMG_HEIGHT, MODEL_SCALE

test_transform = torchvision.transforms.Compose([
    CropBottomHalf(),
    PadByMean(),
    Resize(IMG_WIDTH, IMG_HEIGHT),
    Normalize(),
    ToCHWOrder()
])

In [ ]:
from pku_autonomous_driving.dataset import CarDataset, create_data_loader

test = io.load_test_data()

test_dataset = CarDataset(test, training=False, transform=test_transform)
test_loader = create_data_loader(test_dataset, batch_size=1)

In [ ]:
base_model = resnet.resnext50_32x4d(pretrained=False)
model = centernet.CentResnet(base_model, 8)

setup_kwargs = {"model": model, "device": device}
if 'INITIAL_WEIGHTS' in os.environ:
    setup_kwargs["path"] = Path(os.environ["INITIAL_WEIGHTS"])
util.setup_model(**setup_kwargs)

In [ ]:
%%time
predictions = prediction.predict(model, test_loader, device)

In [ ]:
test = io.load_test_data_raw()
test['PredictionString'] = predictions
test.to_csv('predictions.csv', index=False)
test.head()